# 5장

## 5.1 계산 그래프

### 5.1.1 계산 그래프로 풀다

### 5.1.2 국소적 계산

### 5.1.3 왜 계산 그래프로 푸는가?

1. 전체가 아무리 복잡해도 각 노드에서는 단순 계산만 하면 된다
2. 중간 계산 결과를 모두 보관 가능. 
(o. 장점일까? 이것으로 인한 암시적 메모리 기능이 있는데, neural turing machine 같은경우 그런 메모리를 분리하는 듯.)
3. 진짜 이유. 역전파를 통해 '미분'을 효율적으로 계산하기 위해.

(궁금. 미분을 계산하면 각 노드가 얼마나 최종값을 바꾸는지 알 수 있겠지만, 실제 쓰는건 손실함수인데 그게 정확히 어떻게 연계되는거지?)

## 5.2 연쇄법칙

### 5.2.2 연쇄법칙이란?

> 연쇄법칙: 합성함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

### 5.2.3 연쇄법칙과 계산 그래프

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

`z=x+y` 에서,
`pdz/pdx=1` & `pdz/pdy=1`

### 5.3.2 곱셈 노드의 역전파

`z=xy`에서,
`pdz/pdx=y` & `pdz/pdy=x`


## 5.4 단순한 계층 구현하기


In [10]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None
    
    def foward(self,x,y):
        self.x=x
        self.y=y
        
    def backward(self,dout):
        dx=dout*self.y
        dy=dout*self.x
        return dx,dy    
    
class AddLayer:
    def __init_(self):
        pass
    
    def forward(self, x,y):
        out=x+y
        return out
    
    def backward(self, out):
        dx=dout*1
        dy=dout*1
        return dx,dy
    

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

활성화 함수 ReLU의 수식:
    
\begin{gather*}
y=\begin{cases}
x & ( x >0)\\
0 & ( x\leqslant 0)
\end{cases}\\
\end{gather*}

그의 미분:
    
\begin{gather*}
\frac{\partial y}{\partial x} =\begin{cases}
1 & ( x >0)\\
0 & ( x\leqslant 0)
\end{cases}\\
\end{gather*}


### 5.5.2 Sigmoid 계층

활성화 함수 Sigmoid의 수식:

\begin{gather*}
y\ =\ \frac{1}{1+e^{-x}}
\end{gather*}

그의 미분:

\begin{gather*}
y\ =\ \frac{1}{1+e^{-x}}\\
\\
\frac{\partial L}{\partial y} y^{2} e^{-x} =\frac{\partial L}{\partial y} y( 1-y)
\end{gather*}

구현:


In [5]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out=None
        
    def forward(self,x):
        out=1/(1+np.exp(-x))
        self.out=out
        
    def backward(self,dout):
        dx=dout*(1.0-self.out)*self.out
        return dx

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층



In [6]:
import numpy as np

X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)
Y = np.dot(X,W)+B
print(Y)

(2,)
(2, 3)
(3,)
[0.34060857 1.16929798 1.66637798]


신경망 순전파시에 위처럼 행렬곱으로 계산했다.
역전파시에는 아래와 같은 식이 도출된다.

\begin{gather*}
\frac{\partial L}{\partial X} =\frac{\partial L}{\partial Y} \cdotp W^{T}\\
\\
\frac{\partial L}{\partial W} =X^{T} \cdotp \frac{\partial L}{\partial Y}
\end{gather*}

*(잘 이해안됨..)*

### 5.6.2 배치용 Affine 계층

X 하나에 대한 것이 아닌, X묶음인 배치에 대한 계산

아래는 Affine 구현

In [1]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None
        
    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b
        return out
    
    def backward(self,dout):
        dx=np.dot(dout,self.W.T)
        self.dW=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)
        return dx

### 5.6.3 Softmax-with-Loss 계층

p.176
신경망은 추론 기능, 학습 기능이 있는데, 추론할 때는 Softmax가 필요하지 않지만, 학습할때는 Softmax계층을 사용한다

Softmax-with-Loss계층은 Softmax+Cross Entropy Error 계층을 붙인것



In [2]:
import numpy as np

class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None
        
    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y-self.t)/batch_size
        
        return dx

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림

0. 전제: 신경망의 학습이란, 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정
1. 미니배치: 훈련 데이터 중 일부를 무작위로 가져온다(미니배치)
2. 기울기 산출: 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.
3. 매개변수 갱신: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
4. 반복

사실 수치 미분으로도 기울기를 계산할 수 있으나 느리다. 그래서 오차역전파법을 사용하는 것.

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

